In [11]:
import gzip
from spacy.language import Language
from spacy.lang.char_classes import LIST_ELLIPSES, LIST_CURRENCY, LIST_QUOTES, LIST_PUNCT
from spacy.tokenizer import Tokenizer
from spacy.util import registry, compile_prefix_regex, compile_infix_regex, compile_suffix_regex
from spacy.vocab import Vocab


kbd_vocab = Vocab().from_bytes(gzip.open('../data/processed/spacy/kbd_with_vectors.bin.gz', 'rb').read())

@registry.languages("kbd")
class KbdLanguage(Language):
    lang = "kbd"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        self.vocab = kbd_vocab
        
        # Настройка регулярных выражений для токенизатора
        infix_re = compile_infix_regex(LIST_ELLIPSES + LIST_CURRENCY + LIST_QUOTES + LIST_PUNCT)
        prefix_re = compile_prefix_regex(self.Defaults.prefixes)
        suffix_re = compile_suffix_regex(self.Defaults.suffixes)

        # Настройка токенизатора для нового языка
        self.tokenizer = Tokenizer(
            self.vocab,
            prefix_search=prefix_re.search,
            suffix_search=suffix_re.search,
            infix_finditer=infix_re.finditer,
            token_match=None  # Здесь можно добавить собственные правила для совпадения токенов
        )


In [12]:
from spacy.language import Language
from spacy.tokens import Token

# Регистрируем кастомные атрибуты для токенов Spacy 
Token.set_extension('custom_prefix', default=None, force=True)
Token.set_extension('custom_suffix', default=None, force=True)
Token.set_extension('custom_stem', default=None, force=True)

from tokenizers import Tokenizer as HFTokenizer

tokenizer_uni = HFTokenizer.from_file('../data/processed/tokenizer/words_unigram_5000.tokenizer.json')

@Language.factory("custom_prefix_suffix")
def create_custom_prefix_suffix_component(nlp, name):
    def custom_prefix_suffix(doc):
        for _token in doc:
            word_tokens = tokenizer_uni.encode(_token.text).tokens
            _token._.custom_prefix = word_tokens[0] if len(word_tokens) > 0 else None
            _token._.custom_suffix = word_tokens[-1] if len(word_tokens) > 1 else None
            _token._.custom_stem = ''.join(word_tokens[1:-1]) if len(word_tokens) > 2 else None

        return doc
    return custom_prefix_suffix

In [13]:
from spacy.language import Language

verb_suffixes = ["ыж", "ын", "ащ"]  # Пример: список окончаний глаголов

@Language.factory("simple_pos_tagger")
def create_simple_pos_tagger(nlp, name):
    def simple_pos_tagger(doc):
        # Пример простого правила: присваиваем тег "NOUN" (существительное), если слово заканчивается на "ность"
        for token in doc:
            for suffix in verb_suffixes:
                if token.text.endswith(suffix):
                    token.pos_ = "VERB"  # Глагол
                    break
            
            # Добавьте здесь другие правила
            else:
                token.pos_ = "X"  # Неизвестная часть речи
        return doc
    return simple_pos_tagger

In [14]:
from spacy.pipeline import Lemmatizer

LEMMA_INDEX = {
    "къиcтхыкIащ": "къитхыкIын",
    "къритхыкIынущ": "къитхыкIын",
}

# искдючения из правил, для которых не будет применятся LEMMA_RULES
LEMMA_EXC = {
    
}


LEMMA_RULES = {
    "verbs": [
        ["ыж", "ыжын"],  # Удалить "ать" в конце глаголов
    ],
    "nouns": [
        # Правила для существительных
    ]
    # Добавьте другие правила по аналогии
}


class KbdLanguageLemmatizer(Lemmatizer):
    def __init__(self, vocab, name="kbd_lemmatizer", mode="rule"):
        # Обязательно вызываем конструктор базового класса
        super().__init__(vocab, model=None, name=name, mode=mode)
        self.lemma_index = LEMMA_INDEX
        self.lemma_exc = LEMMA_EXC
        self.lemma_rules = LEMMA_RULES

    def __call__(self, doc):
        # Пройдемся по всем токенам в документе и присвоим леммы
        for token in doc:
            lemma = self.lemmatize(token)  # Используйте здесь свою логику лемматизации
            token.lemma_ = lemma[0] if lemma else token.text  # Присвоим лемму токену
        return doc
    
    def rule_lemmatize(self, token):
        token_text = token.text

        # поиск в индексе
        if token.text in self.lemma_index:
            return [self.lemma_index[token_text]]

        # исключения    
        if token.text in self.lemma_exc:
            return None
        
        if token.pos_ == "VERB":
            # замена окончания
            for old, new in self.lemma_rules["verbs"]:
                if token_text.endswith(old):
                    return [token_text[: -len(old)] + new]
        
        return [token_text]


@Language.factory("kbd_lemmatizer")
def create_kbd_language_lemmatizer(nlp, name):
    return KbdLanguageLemmatizer(nlp.vocab, name=name, mode="rule")


In [15]:
import spacy
nlp = spacy.blank("kbd")
nlp.initialize()

len(nlp.vocab.strings)

784764

In [16]:

# Добавляем нашу функцию в пайплайн
nlp.add_pipe("custom_prefix_suffix")
nlp.add_pipe("simple_pos_tagger")
nlp.add_pipe("kbd_lemmatizer")


In [17]:
doc = nlp("къитхыкIыж къиcтхыкIащ къритхыкIынущ")
for token in doc:
    print('|'.join([token.text, token._.custom_prefix, token._.custom_stem, token._.custom_suffix, token.lemma_, token.pos_, token.tag_]))

къитхыкIыж|къ|итхы|кIыж|къитхыкIыжын|VERB|
къиcтхыкIащ|къ|иcтхыкI|ащ|къитхыкIын|VERB|
къритхыкIынущ|къ|ритхыкI|ынущ|къитхыкIын|X|


In [18]:
'къритхыкIынущ' in nlp.vocab.strings

True

In [20]:
nlp.vocab['къитхыкIыж'].has_vector

True